# 업무 자동화 
 한글파일 이력서 정리하기

In [1]:
# 한글파일에서 원하는 데이터 추출하기
def result(data):
    data = data.split('><')
    data_out = [ ]
    df = ['성          명' , '생  년  월  일', '현    주    소', '연    락    처','이    메    일']
    for i in range(len(data)):
        for j in range(len(df)):
            if data[i] == '이    메    일':
                a = data[i+1].find('>')
                data_out.append(data[i+1][:a])
                break
            if data[i] == df[j]:
                data_out.append(data[i+1])
    df1 = ['학교명','전공','성적','졸업구분','소재지']
    for i in range(len(data)):
        for j in range(len(df1)):
            if data[i] == '성적':
                if data[i+5] != '':
                    score = data[i+5]
                    score = score.replace('/4.5','')
                    score = float(score)
                    if score >=4.3 : 
                        data_out.append(10)
                        break
                    elif 4.3>score>=4.1 :
                        data_out.append(8)
                        break
                    elif 4.1>score>=3.9 :
                        data_out.append(6)
                        break
                    else :
                        data_out.append(4)
                        break
                else:
                    break 
            if data[i] == '소재지':
                b = data[i+5].find('>')
                data_out.append(data[i+5][:b])
                break
            if data[i] ==df1[j]:
                data_out.append(data[i+5])
    for i in range(len(data)):
        if data[i] == 'TOEIC':
            toeic_score = data[i+5]
    if toeic_score != '':
        toeic_score = int(toeic_score.replace('점', ''))

        if toeic_score >900 :
            data_out.append(10)
        elif 900 >= toeic_score >800 : 
            data_out.append(8)
        elif 800 >= toeic_score >700 : 
            data_out.append(6)
        else : 
            data_out.append(4)
    tot = []
    i = data.index('점수')
    while True:
        i += 1
        if data[i] != '기간':
            tot.append(data[i])
        else: 
            break
    del tot[3]
    list_1 = []
    for i in range(0,len(tot),3):
        list_1.append(tot[i])
    score = 0
    for i in range(len(list_1)) :
        if len(list_1) <= 0:
            break 
        else :
            if list_1[i].find('기사') == 4:
                 score += 5
            else : 
                score += 1
    data_out.append(score)
    tot = []
    i = data.index('군필여부')
    while True:
        i += 1
        if data[i] != '군별':
            tot.append(data[i])
        else: 
            break
    data_out.append(int((len(tot) -1) / 2))
    df3 = ['군필여부','군별','계급','복무기간']
    for i in range(len(data)):
        for j in range(len(df3)):
            if data[i] == '복무기간':
                a = data[i+1].find('>')
                data_out.append(data[i+1][:a])
                break
            if data[i] == df3[j]:
                data_out.append(data[i+1])
    df4 = [ 'R','Python','Javascript','SQL','HTML/CSS']
    test = []
    for i in range(len(data)):
        for j in range(len(df4)):
            if data[i] == 'HTML/CSS':
                a = data[i+5].find('>')
                test.append(data[i+4][:a])
                break
            if data[i] == df4[j]:
                test.append(data[i+5])
    score1 = 0
    for i in test :
        if i == '상' :
            score1 +=2
        elif i == '중' :
            score1 += 1
        else : 
            score += 0
    data_out.append(score1)

    return data_out


In [2]:
# 출력해보기
import ole 

with ole.open('./data/이력서_1.hwp') as f:
    data = f.get_stream('PrvText').read().decode('utf-16le')
    
result(data)

['홍길동',
 '1993년 8월 13일',
 '제주특별자치도 제주시',
 '010-2233-1478',
 'ab8ab@naver.com',
 '건국대학교',
 '컴퓨터공학과',
 8,
 '졸업',
 '서울',
 6,
 6,
 1,
 '군필',
 '육군',
 '병장',
 '15.02~16.11',
 4]

In [7]:
# 총 100개의 hwp 파일 => 엑셀에 저장하기
import glob
import xlsxwriter
import numpy as np
import pandas as pd
import ole
path = ('./data/*.hwp')
workbook = xlsxwriter.Workbook('tutorial.xlsx')
worksheet = workbook.add_worksheet()
num = 1
# 각 한글 폴더에서 성명 , 소속, 연락처 ,e-mail 만 추출  
df = ['이름' , '생년월일' , '주소' , '번호' , '이메일', '대학교', '전공' ,
      '학점' , '졸업구분' , '소재지' , 'TOEIC' , '자격증' ,
      '교육/활동/수상' ,'군필여부', '군별', '계급' ,'복무기간' , 
      '컴퓨터 언어 능력']
for n,i in enumerate(df):
    worksheet.write(0,n ,i)

for fname in glob.glob(path):
       with ole.open(fname) as f:
            data = f.get_stream('PrvText').read().decode('utf-16le')
            data = result(data)
            for number,values in enumerate(data):
                worksheet.write(num ,number, values)
            num += 1

workbook.close()      